In [28]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error, r2_score
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
df = pd.read_csv('/content/drive/My Drive/Resources/kingcounty/seattle_sales_cleaned_data.csv')
# Review the DataFrame
df


,id,sale_date,sale_price,latitude,longitude,city,zoning,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,bath_3qtr,bath_half,garb_sqft,gara_sqft,wfnt,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,year
0,0,1/2/2018,665000,47.56,-122.32,SEATTLE,SF 5000,41000,122000,1920,2525,1570,6,4,4,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018
1,1,1/9/2018,499000,47.52,-122.32,SEATTLE,SF 5000,29000,55000,1907,6000,1350,5,0,3,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018
2,2,2/14/2018,675000,47.62,-122.30,SEATTLE,SF 5000,97000,132000,1908,5286,1800,7,5,4,1.00,4,3,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2018
3,3,2/13/2018,1150000,47.72,-122.34,SEATTLE,SF 7200,440000,221000,1920,47916,1590,7,0,3,1.50,3,1,0,0,0,0,8,0,0,2,0,0,0,0,0,0,0,4,2018
4,4,2/14/2018,705000,47.67,-122.32,SEATTLE,SF 5000,81000,72000,1918,3825,780,6,0,3,1.00,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24933,24945,12/8/2022,895000,47.69,-122.33,SEATTLE,NR3,440000,327000,1926,3000,1100,7,7,3,1.50,3,2,0,0,370,0,0,0,0,0,0,0,0,0,0,0,0,0,2022
24934,24946,12/5/2022,1900000,47.60,-122.29,SEATTLE,NR3,790000,1432000,1924,5016,3000,10,8,3,2.00,4,2,1,0,240,0,0,0,0,0,0,0,2,2,0,0,1,0,2022
24935,24947,12/17/2022,1750000,47.63,-122.31,SEATTLE,NR3,853000,727000,1908,4800,2900,9,8,3,1.50,4,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2022
24936,24948,12/15/2022,1250000,47.68,-122.39,SEATTLE,NR3,497000,532000,1928,3825,1730,8,8,3,1.00,3,1,1,0,180,0,0,0,0,0,0,0,0,0,0,0,0,0,2022


In [30]:
df['sale_date'] = pd.to_datetime(df['sale_date'])
df['sale_month'] = df['sale_date'].dt.month
df['sale_day'] = df['sale_date'].dt.day
df.rename(columns={'year': 'sale_year'}, inplace=True)
df.head()

,id,sale_date,sale_price,latitude,longitude,city,zoning,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,bath_3qtr,bath_half,garb_sqft,gara_sqft,wfnt,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,sale_year,sale_month,sale_day
0,0,2018-01-02,665000,47.56,-122.32,SEATTLE,SF 5000,41000,122000,1920,2525,1570,6,4,4,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1,2
1,1,2018-01-09,499000,47.52,-122.32,SEATTLE,SF 5000,29000,55000,1907,6000,1350,5,0,3,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1,9
2,2,2018-02-14,675000,47.62,-122.30,SEATTLE,SF 5000,97000,132000,1908,5286,1800,7,5,4,1.00,4,3,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2018,2,14
3,3,2018-02-13,1150000,47.72,-122.34,SEATTLE,SF 7200,440000,221000,1920,47916,1590,7,0,3,1.50,3,1,0,0,0,0,8,0,0,2,0,0,0,0,0,0,0,4,2018,2,13
4,4,2018-02-14,705000,47.67,-122.32,SEATTLE,SF 5000,81000,72000,1918,3825,780,6,0,3,1.00,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,2,14


In [32]:
mil_df = df[df["sale_price"] >= 1000000]
mil_df

,id,sale_date,sale_price,latitude,longitude,city,zoning,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,bath_3qtr,bath_half,garb_sqft,gara_sqft,wfnt,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,sale_year,sale_month,sale_day
3,3,2018-02-13,1150000,47.72,-122.34,SEATTLE,SF 7200,440000,221000,1920,47916,1590,7,0,3,1.50,3,1,0,0,0,0,8,0,0,2,0,0,0,0,0,0,0,4,2018,2,13
9,9,2018-02-20,1760000,47.62,-122.30,SEATTLE,SF 5000,49000,97000,1942,5500,1360,7,0,3,1.00,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,2,20
14,14,2018-03-14,1265500,47.67,-122.30,SEATTLE,SF 5000,45700,83700,1925,3700,1250,7,0,3,1.50,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,3,14
15,15,2018-03-28,1168000,47.69,-122.36,SEATTLE,SF 5000,70000,118000,1980,5334,2400,6,0,3,2.00,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,3,28
16,16,2018-03-16,2345000,47.63,-122.36,SEATTLE,SF 5000,85000,39100,1904,4800,1420,6,0,3,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,3,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24931,24943,2022-11-23,2850000,47.59,-122.38,SEATTLE,NR3,1052000,1318000,1991,3960,3220,10,9,3,2.00,4,2,1,1,480,0,0,0,0,0,0,0,4,4,4,4,0,0,2022,11,23
24934,24946,2022-12-05,1900000,47.60,-122.29,SEATTLE,NR3,790000,1432000,1924,5016,3000,10,8,3,2.00,4,2,1,0,240,0,0,0,0,0,0,0,2,2,0,0,1,0,2022,12,5
24935,24947,2022-12-17,1750000,47.63,-122.31,SEATTLE,NR3,853000,727000,1908,4800,2900,9,8,3,1.50,4,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2022,12,17
24936,24948,2022-12-15,1250000,47.68,-122.39,SEATTLE,NR3,497000,532000,1928,3825,1730,8,8,3,1.00,3,1,1,0,180,0,0,0,0,0,0,0,0,0,0,0,0,0,2022,12,15


In [33]:
mil_df['zoning'].value_counts()
category_mapping = {
    'NR3': 1,
    'NR2': 2,
    'SF 5000': 3,
    'RSL (M)': 4,
    'SF 7200': 5,
    'NR1': 6,
    'SF 9600': 7
}

# Replace categorical values with integer values using the mapping
mil_df['zoning'] = mil_df['zoning'].map(category_mapping)
mil_df['zoning'].value_counts()

<ipython-input-33-1e2b2ef6574d>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mil_df['zoning'] = mil_df['zoning'].map(category_mapping)


1    6486
3    1451
2    1204
6     234
5     198
4     190
7      45
Name: zoning, dtype: int64

In [34]:
mil_df['sale_date'] = pd.to_datetime(df['sale_date'])
mil_df['sale_date'] = mil_df['sale_date'].astype('int64') // 10**9
mil_df['sale_date']

<ipython-input-34-bfb7c721e7c5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mil_df['sale_date'] = pd.to_datetime(df['sale_date'])
<ipython-input-34-bfb7c721e7c5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mil_df['sale_date'] = mil_df['sale_date'].astype('int64') // 10**9


3        1518480000
9        1519084800
14       1520985600
15       1522195200
16       1521158400
            ...    
24931    1669161600
24934    1670198400
24935    1671235200
24936    1671062400
24937    1671148800
Name: sale_date, Length: 9808, dtype: int64

In [35]:
mil_df = mil_df.drop(columns=['city'])
mil_df

,id,sale_date,sale_price,latitude,longitude,zoning,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,bath_3qtr,bath_half,garb_sqft,gara_sqft,wfnt,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,sale_year,sale_month,sale_day
3,3,1518480000,1150000,47.72,-122.34,5,440000,221000,1920,47916,1590,7,0,3,1.50,3,1,0,0,0,0,8,0,0,2,0,0,0,0,0,0,0,4,2018,2,13
9,9,1519084800,1760000,47.62,-122.30,3,49000,97000,1942,5500,1360,7,0,3,1.00,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,2,20
14,14,1520985600,1265500,47.67,-122.30,3,45700,83700,1925,3700,1250,7,0,3,1.50,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,3,14
15,15,1522195200,1168000,47.69,-122.36,3,70000,118000,1980,5334,2400,6,0,3,2.00,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,3,28
16,16,1521158400,2345000,47.63,-122.36,3,85000,39100,1904,4800,1420,6,0,3,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,3,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24931,24943,1669161600,2850000,47.59,-122.38,1,1052000,1318000,1991,3960,3220,10,9,3,2.00,4,2,1,1,480,0,0,0,0,0,0,0,4,4,4,4,0,0,2022,11,23
24934,24946,1670198400,1900000,47.60,-122.29,1,790000,1432000,1924,5016,3000,10,8,3,2.00,4,2,1,0,240,0,0,0,0,0,0,0,2,2,0,0,1,0,2022,12,5
24935,24947,1671235200,1750000,47.63,-122.31,1,853000,727000,1908,4800,2900,9,8,3,1.50,4,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2022,12,17
24936,24948,1671062400,1250000,47.68,-122.39,1,497000,532000,1928,3825,1730,8,8,3,1.00,3,1,1,0,180,0,0,0,0,0,0,0,0,0,0,0,0,0,2022,12,15


In [36]:
mil_df = mil_df.drop(columns=['sale_date'])

In [37]:
mil_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9808 entries, 3 to 24937
Data columns (total 35 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                9808 non-null   int64  
 1   sale_price        9808 non-null   int64  
 2   latitude          9808 non-null   float64
 3   longitude         9808 non-null   float64
 4   zoning            9808 non-null   int64  
 5   land_val          9808 non-null   int64  
 6   imp_val           9808 non-null   int64  
 7   year_built        9808 non-null   int64  
 8   sqft_lot          9808 non-null   int64  
 9   sqft              9808 non-null   int64  
 10  grade             9808 non-null   int64  
 11  fbsmt_grade       9808 non-null   int64  
 12  home_condition    9808 non-null   int64  
 13  stories           9808 non-null   float64
 14  beds              9808 non-null   int64  
 15  bath_full         9808 non-null   int64  
 16  bath_3qtr         9808 non-null   int64  

In [38]:
summary_stats = mil_df.describe()
print(summary_stats)

            id  sale_price  latitude  longitude  zoning   land_val    imp_val  \
count  9808.00     9808.00   9808.00    9808.00 9808.00    9808.00    9808.00   
mean  13480.66  1591453.42     47.64    -122.34    1.70  635888.91  662636.54   
std    8003.88   780888.49      0.05       0.05    1.22  468082.91  457031.49   
min       3.00  1000000.00     47.50    -122.42    1.00       0.00       0.00   
25%    5948.25  1157500.00     47.62    -122.38    1.00  386000.00  381000.00   
50%   15070.50  1365000.00     47.65    -122.34    1.00  586000.00  614500.00   
75%   20411.25  1750000.00     47.68    -122.29    2.00  792000.00  859000.00   
max   24949.00 14850000.00     47.73    -122.24    7.00 7560000.00 7946000.00   

       year_built  sqft_lot     sqft   grade  fbsmt_grade  home_condition  \
count     9808.00   9808.00  9808.00 9808.00      9808.00         9808.00   
mean      1942.63   6337.45  2420.84    7.97         5.11            3.79   
std         31.09   4648.03   956.69   

In [39]:
columns = ['sale_price', 'sqft_lot', 'sqft', 'land_val', 'imp_val']

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Calculate IQR for the specified columns
Q1 = mil_df[columns].quantile(0.25)
Q3 = mil_df[columns].quantile(0.75)
IQR = Q3 - Q1

# Define lower and upper bounds for the specified columns
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter outliers for the specified columns
filtered_df = mil_df[~((mil_df[columns] < lower_bound) | (mil_df[columns] > upper_bound)).any(axis=1)]
filtered_df = filtered_df.reset_index(drop=True)
filtered_df.head()

,id,sale_price,latitude,longitude,zoning,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,bath_3qtr,bath_half,garb_sqft,gara_sqft,wfnt,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,sale_year,sale_month,sale_day
0,9,1760000,47.62,-122.30,3,49000,97000,1942,5500,1360,7,0,3,1.00,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,2,20
1,14,1265500,47.67,-122.30,3,45700,83700,1925,3700,1250,7,0,3,1.50,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,3,14
2,15,1168000,47.69,-122.36,3,70000,118000,1980,5334,2400,6,0,3,2.00,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,3,28
3,16,2345000,47.63,-122.36,3,85000,39100,1904,4800,1420,6,0,3,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,3,16
4,18,1370000,47.68,-122.35,3,78000,121000,1908,4000,1320,6,0,3,1.50,2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,4,16


In [40]:
y = filtered_df["sale_price"].ravel()
scaler = StandardScaler()
y_scaled = scaler.fit_transform(y.reshape(-1, 1)).ravel()
X = filtered_df.copy()
X.drop("sale_price", axis=1, inplace=True)

In [41]:
y_scaled[:5]

array([ 0.94282894, -0.42548344, -0.69527203,  2.56156047, -0.13632541])

In [42]:
X.dropna()
X.head()

,id,latitude,longitude,zoning,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,bath_3qtr,bath_half,garb_sqft,gara_sqft,wfnt,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,sale_year,sale_month,sale_day
0,9,47.62,-122.30,3,49000,97000,1942,5500,1360,7,0,3,1.00,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,2,20
1,14,47.67,-122.30,3,45700,83700,1925,3700,1250,7,0,3,1.50,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,3,14
2,15,47.69,-122.36,3,70000,118000,1980,5334,2400,6,0,3,2.00,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,3,28
3,16,47.63,-122.36,3,85000,39100,1904,4800,1420,6,0,3,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,3,16
4,18,47.68,-122.35,3,78000,121000,1908,4000,1320,6,0,3,1.50,2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,4,16


In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, random_state=42)

In [44]:
regression_model = LinearRegression()

# Fit the model using training data
regression_model.fit(X_train, y_train)

# Predict values using the trained model
y_pred = regression_model.predict(X_test)

In [45]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 0.7295173876331145
R-squared: 0.2950185261870787


In [46]:
rf_model = RandomForestRegressor(n_estimators=200, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=200, random_state=42)

In [47]:
y_pred2 = rf_model.predict(X_test)

In [48]:
pd.options.display.float_format = '{:.2f}'.format
y_pred2_original = scaler.inverse_transform(y_pred2.reshape(-1, 1)).ravel()

# Inverse transform the scaled actual prices to get original prices
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).ravel()

mse = mean_squared_error(y_test_original, y_pred2_original)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_original, y_pred2_original)
rmse_rounded = round(rmse) # Round and convert to integer

absolute_diff = np.abs(y_test_original - y_pred2_original)

# Create a DataFrame for comparison
comparison_df = pd.DataFrame({
    'Actual Price': y_test_original,
    'Predicted Price': y_pred2_original,
    'Absolute Difference': absolute_diff
})

# Print the comparison DataFrame
print(comparison_df)
print("Root Mean Squared Error:", rmse_rounded)
print("R-squared:", r2)

      Actual Price  Predicted Price  Absolute Difference
0       1647000.00       1461961.00            185039.00
1       1075000.00       1132522.88             57522.88
2       1114855.00       1219552.32            104697.32
3       1185000.00       1339633.03            154633.03
4       1148000.00       1587393.09            439393.09
...            ...              ...                  ...
1690    1310000.00       1416390.07            106390.07
1691    1262000.00       1250631.27             11368.73
1692    1185000.00       1224805.20             39805.20
1693    1735000.00       1717839.25             17160.75
1694    1285000.00       1135377.27            149622.73

[1695 rows x 3 columns]
Root Mean Squared Error: 206858
R-squared: 0.6833890750601006
